In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:

path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB053210,Karma Resorts,Other,Past Guests,Karma Bavaria,PF328596,Mr,Simon,Wilson,Male,...,2025-09-15 00:00:00,7,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email Only,NO
1,KK076588,Karma Resorts,Other,Past Guests,Karma Kandara,PF320020,Miss,Robyn Anne,Whalan,Female,...,2025-09-15 00:00:00,2,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
2,KK074398,Karma Resorts,Other,Past Guests,Karma Kandara,PF314251,Mr.,Malcolm Stephen,Watkins,Male,...,2025-09-18 00:00:00,3,HOUSE_USE,V_OWNER,COMP,5,0,0,Email and Phone,NO
3,KK074130,Karma Resorts,Other,Past Guests,Karma Kandara,PF125748,Mr,Kevin,Matthews,Male,...,2025-09-14 00:00:00,13,HOUSE_USE,V_OWNER,COMP,6,0,0,Email and Phone,NO
4,KK079293,Karma Resorts,Other,Past Guests,Karma Kandara,PF267203,Miss,Yena,Choi,Female,...,2025-09-18 00:00:00,4,HOUSE_USE,PRESS,MEDIA,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,KSM014610,Karma Resorts,Other,Past Guests,Karma St Martins,PF300066,<NA>,Christine,Eade,Unknown,...,2025-09-19 00:00:00,4,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
99,KSM014740,Karma Resorts,Other,Past Guests,Karma St Martins,PF303281,Mr,& Mrs,Phillipson,Unknown,...,2025-09-15 00:00:00,2,DIR,TEL,RETAIL,2,0,0,Phone Only,NO
100,KK077277,Karma Resorts,Other,Past Guests,Karma Kandara,PF321996,Mr,Taylor J,Stanley,Male,...,2025-09-17 00:00:00,5,TO,TO,RETAIL,2,0,0,Email and Phone,NO
101,KK079388,Karma Resorts,Other,Past Guests,Karma Kandara,PF328001,Mrs,Jennifer,Elaine Jingco,Female,...,2025-09-17 00:00:00,19,HOUSE_USE,V_OWNER_NRB,COMP,3,2,0,Email and Phone,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [8]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
11,KK069664,Karma Resorts,Other,Past Guests,Karma Kandara,PF299367,Mr,Robyn,Whalan,Male,...,2025-09-15 00:00:00,2,OTA,OTA,RETAIL,1,0,0,Phone Only,NO
25,KB053376,Karma Resorts,Other,Past Guests,Karma Bavaria,PF018070,Herr,Rolf,Badenhoop,Male,...,2025-09-16 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
46,KB047628,Karma Resorts,Other,Past Guests,Karma Bavaria,PF276256,Prof.,Dietrich,Grashoff,Unknown,...,2025-09-14 00:00:00,3,DIR,EMAIL_MRKTNG,DIGITAL,2,0,0,Email and Phone,NO
53,KB053156,Karma Resorts,Other,Past Guests,Karma Bavaria,PF308195,Herr,Peter,Braunagel,Male,...,2025-09-15 00:00:00,1,DIGITAL,TEL,RETAIL,2,0,0,Email and Phone,NO
62,KK067267,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-09-20 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
74,KK066251,Karma Resorts,Other,Past Guests,Karma Kandara,PF295555,Mr.,Gordon Douglas,Mckendry,Male,...,2025-09-16 00:00:00,7,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
76,KK074039,Karma Resorts,Other,Past Guests,Karma Kandara,PF313347,Mr.,Jose Ignacio,Ayala Caceres,Male,...,2025-09-19 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
81,KSM015351,Karma Resorts,Other,Past Guests,Karma St Martins,PF230972,Mr,Nicholas,Williams,Unknown,...,2025-09-18 00:00:00,6,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO


In [9]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB053210,Karma Resorts,Other,Past Guests,Karma Bavaria,PF328596,Mr,Simon,Wilson,Male,...,2025-09-15 00:00:00,7,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email Only,NO
1,KK076588,Karma Resorts,Other,Past Guests,Karma Kandara,PF320020,Miss,Robyn Anne,Whalan,Female,...,2025-09-15 00:00:00,2,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
2,KK074398,Karma Resorts,Other,Past Guests,Karma Kandara,PF314251,Mr.,Malcolm Stephen,Watkins,Male,...,2025-09-18 00:00:00,3,HOUSE_USE,V_OWNER,COMP,5,0,0,Email and Phone,NO
3,KK074130,Karma Resorts,Other,Past Guests,Karma Kandara,PF125748,Mr,Kevin,Matthews,Male,...,2025-09-14 00:00:00,13,HOUSE_USE,V_OWNER,COMP,6,0,0,Email and Phone,NO
4,KK079293,Karma Resorts,Other,Past Guests,Karma Kandara,PF267203,Miss,Yena,Choi,Female,...,2025-09-18 00:00:00,4,HOUSE_USE,PRESS,MEDIA,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,KSM014610,Karma Resorts,Other,Past Guests,Karma St Martins,PF300066,empty,Christine,Eade,Unknown,...,2025-09-19 00:00:00,4,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
91,KSM014740,Karma Resorts,Other,Past Guests,Karma St Martins,PF303281,Mr,& Mrs,Phillipson,Unknown,...,2025-09-15 00:00:00,2,DIR,TEL,RETAIL,2,0,0,Phone Only,NO
92,KK077277,Karma Resorts,Other,Past Guests,Karma Kandara,PF321996,Mr,Taylor J,Stanley,Male,...,2025-09-17 00:00:00,5,TO,TO,RETAIL,2,0,0,Email and Phone,NO
93,KK079388,Karma Resorts,Other,Past Guests,Karma Kandara,PF328001,Mrs,Jennifer,Elaine Jingco,Female,...,2025-09-17 00:00:00,19,HOUSE_USE,V_OWNER_NRB,COMP,3,2,0,Email and Phone,NO


In [10]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [11]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [12]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
3,KK074130,Karma Resorts,Other,Past Guests,Karma Kandara,PF125748,Mr,Kevin,Matthews,Male,...,2025-09-14 00:00:00,13,HOUSE_USE,V_OWNER,COMP,6,0,0,Email and Phone,NO
23,KB047589,Karma Resorts,Other,Past Guests,Karma Bavaria,PF018070,Herr,Rolf,Badenhoop,Male,...,2025-09-15 00:00:00,4,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
24,KB053319,Karma Resorts,Other,Past Guests,Karma Bavaria,PF216954,Mr,David,Raguin,Male,...,2025-09-14 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
36,KB052721,Karma Resorts,Other,Past Guests,Karma Bavaria,PF323967,Mr,Joachim,Schug,Unknown,...,2025-09-17 00:00:00,3,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
37,KB049989,Karma Resorts,Other,Past Guests,Karma Bavaria,PF133894,Mr,Anton,Schwenzer,Male,...,2025-09-16 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
43,KB045796,Karma Resorts,Other,Past Guests,Karma Bavaria,PF276256,Prof.,Dietrich,Grashoff,Unknown,...,2025-09-17 00:00:00,4,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
44,KB050518,Karma Resorts,Other,Past Guests,Karma Bavaria,PF072877,Herr,Dieter,Nees,Male,...,2025-09-20 00:00:00,6,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
57,KK067109,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-09-20 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
86,KSM017041,Karma Resorts,Other,Past Guests,Karma St Martins,PF211895,Mr,Ian,Hancock,Unknown,...,2025-09-17 00:00:00,1,DIR,TEL,RETAIL,3,0,0,Email and Phone,NO


In [13]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [14]:
count_data_akhir = df_analytic.shape[0]

In [15]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB049123,Karma Resorts,Other,Past Guests,Karma Bavaria,PF329679,Frau,Nadine,Hartung,Female,...,2025-09-14 00:00:00,2,LGR,TEL,RETAIL,2,0,0,Email Only,NO
1,KB049122,Karma Resorts,Other,Past Guests,Karma Bavaria,PF308369,Frau,Monika,Richter,Female,...,2025-09-14 00:00:00,2,LGR,TEL,RETAIL,2,0,0,Email and Phone,NO
2,KB049510,Karma Resorts,Other,Past Guests,Karma Bavaria,PF310037,Frau,Michaela,Wulfers,Female,...,2025-09-14 00:00:00,1,LGR,TEL,RETAIL,6,0,0,Email and Phone,NO
3,KB050146,Karma Resorts,Other,Past Guests,Karma Bavaria,PF170539,Frau,Luise,Badenhoop,Female,...,2025-09-14 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
4,KB049867,Karma Resorts,Other,Past Guests,Karma Bavaria,PF243453,Dr.,Julia,Sommer,Female,...,2025-09-14 00:00:00,1,LGR,TEL,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,KK078705,Karma Resorts,Other,Past Guests,Karma Kandara,PF326029,Mr.,Jakob Charles,Zohs,Male,...,2025-09-19 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,6,0,0,Email and Phone,NO
82,KK078039,Karma Resorts,Other,Past Guests,Karma Kandara,PF324304,Mr,Kenji,Sato,Male,...,2025-09-20 00:00:00,1,OTA,OTA,RETAIL,3,0,0,Phone Only,NO
83,KSM016541,Karma Resorts,Other,Past Guests,Karma St Martins,PF324298,Mrs,Claudia,Toth,Female,...,2025-09-20 00:00:00,2,DIR,TEL,RETAIL,2,0,0,Email Only,NO
84,KB050111,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312172,empty,Marlene,Meier,Unknown,...,2025-09-21 00:00:00,1,OTA,OTA,RETAIL,6,0,0,Phone Only,NO


In [16]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [17]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_10796\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [18]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB049123,Karma Resorts,Other,Past Guests,Karma Bavaria,PF329679,Frau,Nadine,Hartung,Female,...,2025-09-14,2,LGR,TEL,RETAIL,2,0,0,Email Only,NO
1,KLM011039,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF318378,Mr,Agency,Chef,Male,...,2025-09-14,2,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
2,KK079983,Karma Resorts,Other,Past Guests,Karma Kandara,PF329701,Mrs,Candace Brit,Sou,Female,...,2025-09-14,1,HOUSE_USE,V_OWNER,COMP,6,2,0,Email and Phone,NO
3,KSM016288,Karma Resorts,Other,Past Guests,Karma St Martins,PF071366,Mr,Richard,Larn,Unknown,...,2025-09-14,7,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
4,KB050196,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312396,Frau,Tanja,Henning,Female,...,2025-09-14,2,LGR,TEL,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,KLM008007,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF299877,,Wilma,Van Weerdenburg,Unknown,...,2025-09-19,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
82,KK078039,Karma Resorts,Other,Past Guests,Karma Kandara,PF324304,Mr,Kenji,Sato,Male,...,2025-09-20,1,OTA,OTA,RETAIL,3,0,0,Phone Only,NO
83,KSM016541,Karma Resorts,Other,Past Guests,Karma St Martins,PF324298,Mrs,Claudia,Toth,Female,...,2025-09-20,2,DIR,TEL,RETAIL,2,0,0,Email Only,NO
84,KB050111,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312172,,Marlene,Meier,Unknown,...,2025-09-21,1,OTA,OTA,RETAIL,6,0,0,Phone Only,NO


In [19]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 103
Jumlah Data Yang Duplicate : 8
Jumlah Data Setelah Hapus Duplicate : 95
Jumlah Data Yang Termasuk Member: 9
Jumlah Data Setelah Hapus Member : 86
Jumlah Data Akhir : 86


In [20]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)